In [3]:
import math
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

In [7]:
#生成数据
max_degree = 20
n_train,n_test = 100,100
true_w = np.zeros(max_degree)
true_w[0:4] = np.array([5,1.2,-4.3,5.6])

X = np.random.normal(size=(n_train + n_test, 1))
np.random.shuffle(X)
X_poly = np.power(X, np.arange(max_degree).reshape(1, -1))
for i in range(max_degree):
    X_poly[:,i] /= math.gamma(i + 1)
y = np.dot(X_poly,true_w)
y += np.random.normal(scale=0.1, size=y.shape)
X,y

(array([[-0.8916256 ],
        [-0.98745839],
        [-0.98445827],
        [-0.94939978],
        [-1.11678546],
        [ 1.06310539],
        [ 1.82620579],
        [-0.69544887],
        [-0.58288487],
        [ 0.34269278],
        [-1.36710075],
        [ 0.05504105],
        [-0.76563437],
        [ 1.13882683],
        [ 2.02099651],
        [ 0.71196409],
        [ 0.77194809],
        [ 0.2138246 ],
        [ 0.23439082],
        [-0.65337184],
        [-0.42604481],
        [-0.96928962],
        [-0.37689268],
        [ 0.48440512],
        [ 1.30421543],
        [ 0.99410448],
        [ 1.00122254],
        [ 1.02805583],
        [-0.8345546 ],
        [ 0.13281487],
        [ 1.62728096],
        [-1.12443113],
        [ 0.90091373],
        [-0.12662073],
        [ 0.65632008],
        [ 1.89541022],
        [-1.04747098],
        [-0.81226764],
        [-0.69512584],
        [-0.58437904],
        [-0.07307764],
        [ 0.10878608],
        [-1.25572187],
        [-0

In [8]:
# NumPy ndarray转换为tensor
true_w, X,  X_poly, y = [torch.tensor(x, dtype=
    torch.float32) for x in [true_w, X, X_poly, y]]

X[:2], X_poly[:2, :], y[:2]

(tensor([[-0.8916],
         [-0.9875]]),
 tensor([[ 1.0000e+00, -8.9163e-01,  3.9750e-01, -1.1814e-01,  2.6334e-02,
          -4.6960e-03,  6.9785e-04, -8.8889e-05,  9.9069e-06, -9.8148e-07,
           8.7511e-08, -7.0934e-09,  5.2705e-10, -3.6149e-11,  2.3022e-12,
          -1.3685e-13,  7.6261e-15, -3.9998e-16,  1.9813e-17, -9.2977e-19],
         [ 1.0000e+00, -9.8746e-01,  4.8754e-01, -1.6047e-01,  3.9615e-02,
          -7.8237e-03,  1.2876e-03, -1.8164e-04,  2.2420e-05, -2.4598e-06,
           2.4290e-07, -2.1805e-08,  1.7943e-09, -1.3629e-10,  9.6129e-12,
          -6.3282e-13,  3.9055e-14, -2.2686e-15,  1.2445e-16, -6.4679e-18]]),
 tensor([1.5006, 0.6380]))

In [9]:
#定义训练函数
def train(X_train, y_train,X_test,y_test,num_epochs = 400): 
    #定义损失函数
    loss = nn.MSELoss()

    input_shape = X_train.shape[-1]
    #定义模型
    net = nn.Sequential(nn.Linear(input_shape,1,bias = false))
    batch_size = Math.min(10, X_train.shape[0])
    train_iter = d2l.load_array((test_features, test_labels.reshape(-1,1)),batch_size, true)
    test_iter = d2l.load_array((test_features, test_labels.reshape(-1,1)),batch_size, false)
    trainer = torch.optim.SGD(net.parameters,lr=0.001)
    #可视化
    animator = d2l.Animator(xlabel='epoch', ylabel='loss', yscale='log',
                        xlim=[1, num_epochs], ylim=[1e-3, 1e2],
                        legend=['train', 'test'])

    #训练 todo
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (epoch + 1) % 20 == 0:
            animator.add(epoch + 1, (d2l.evaluate_loss(net, train_iter, loss),
                                 d2l.evaluate_loss(net, test_iter, loss)))
    print('weight:', net[0].weight.data.numpy())


    



SyntaxError: incomplete input (631503593.py, line 3)